In [42]:
import pandas as pd
import numpy as np
import math
from calendar import monthrange
import warnings
warnings.filterwarnings("ignore")
from datetime import date

In [43]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format

In [44]:
currentYear = 2024
currentMonth = 3
values = {}
values['FY_v'] = False

In [45]:
# Base de dados completa
arq= pd.ExcelFile('Databases/'+ str(currentYear) + "_Commercial_Tool_Database.xlsm")

In [46]:
# General Input Database 
print("Reading Commercial Inputs Database")
#window.refresh() if window else None
sheets= ["Budget","Forecast3+9","Forecast6+6","Forecast9+3","ACT_"+str(currentYear),"LY_"+str(currentYear-1)]
dbs=[]
for s in sheets:
    db= pd.read_excel(arq,s)
    db["Type"]=s
    dbs.append(db)
dfinal = pd.concat(dbs)

Reading Commercial Inputs Database


In [47]:
# Projection Database
print("Starting Projection calculation")
#window.refresh() if window else None
dclientes= pd.read_excel(arq,'Icestar_Clients') # Clientes
dproj= pd.read_excel(arq,'Volumes_Projection')
dprojservs= pd.read_excel(arq,'Services_List')
dproj.rename(columns={'Month': "Mes"}, inplace=True)
dproj.rename(columns={'Week': "Semana"}, inplace=True)
dproj.rename(columns={'BLAST FREEZE': "FREEZE"}, inplace=True)
dproj= dproj.fillna(0)
# Promedio Mensal Saldos ( Monthly Average Storage Balance )
dproj["PromedioM_Frozen"]=0.0
dproj["PromedioM_Refrigerated"]=0.0
dproj["PromedioM_Dry"]=0.0
# Ingrssos Mensal
dproj["Ingressos($)_Frozen"]=0.0
dproj["Ingressos($)_Refrigerated"]=0.0
dproj["Ingressos($)_Dry"]=0.0
sems_meses= dproj.groupby(['Mes']).apply(lambda x: len(x["Semana"].unique())).to_dict()
m = currentMonth-1
clientes= dproj.Client_Code.unique()
meses= dproj[dproj.Mes>m].Mes.unique()
sems= dproj[dproj.Mes>0].Semana.unique()
Storage_Types=["Frozen","Refrigerated","Dry"]
Storage_Types_Dict={"Frozen":"Storage Frozen","Refrigerated":"Storage Refrigerated","Dry":"Storage Dry"}
for f in dproj.Facility.unique():
    print(f)
    #window.refresh() if window else None
    for stt in Storage_Types:
        PromedioMT= "PromedioM_"+ stt
        SaldoS= "Balance(" + stt +")"
        print('Gathering', stt, 'data for months: ', end = " ")
        for m in meses:
            print(m, end = " ")
            #window.refresh() if window else None
            for c in clientes:
                dpm= dproj[(dproj.Facility == f)&(dproj.Mes == m) & (dproj.Client_Code ==c)]
                dproj.loc[(dproj.Facility == f)&(dproj.Mes == m) & (dproj.Client_Code ==c), PromedioMT] = dpm[(dpm.Mes == m )][SaldoS].mean()
        print('')

Starting Projection calculation
Buin
Gathering Frozen data for months:  3 4 5 6 7 8 9 10 11 12 
Gathering Refrigerated data for months:  3 4 5 6 7 8 9 10 11 12 
Gathering Dry data for months:  3 4 5 6 7 8 9 10 11 12 
Quilicura
Gathering Frozen data for months:  3 4 5 6 7 8 9 10 11 12 
Gathering Refrigerated data for months:  3 4 5 6 7 8 9 10 11 12 
Gathering Dry data for months:  3 4 5 6 7 8 9 10 11 12 


In [48]:
# Ingressos en pesos (Storage projection calculation in revenue)
df= pd.read_excel(arq, "Rates")
ufs= pd.read_csv("Databases/External/UF_"+str(currentYear)+".csv",sep=';')
ufs = ufs.melt(id_vars=["Día"], var_name="Mes", value_name="Total")
meses= ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep','Oct', 'Nov', 'Dic']
i=1
for m in meses:
    ufs.loc[(ufs['Mes']== m),"Mes"]=i
    i+=1

dftp = pd.read_excel(arq, "Contracts") # contratos
dftp = dftp.fillna(0)
df = df.fillna(0)
dproj= dproj.fillna(0)
dftp.rename(columns={'Fixed Positions Rate': "Rate"}, inplace=True)
dftp["Variable Positions Rate"]= dftp["Variable Positions Rate"].fillna(0)
# Sem minímo
ufs['Total'] = ufs['Total'].str.replace(".", "")
ufs['Total'] = ufs['Total'].str.replace(",", ".").astype(float)
ufs= ufs.groupby(['Mes']).apply(lambda x: x["Total"].mean()).reset_index(name ='Total')
ufs.loc[(ufs.Mes > currentMonth), 'Total'] = ufs[ufs.Mes == currentMonth]["Total"].sum()
dproj= dproj[dproj.Client_Code!=0]

In [49]:
stt = 'Frozen'
f = 'Quilicura'
clientes_tarifas_base= df[(df.Facility==f)&(df["IceStar Contract Type"]==Storage_Types_Dict[stt])].groupby(['Client_Code',"Moneda_Geral"]).apply(lambda x: x["Valor"].mean()).to_dict()

In [50]:
for f in dftp.Facility.unique():
    print('Running storage calculation for:',f)
    #window.refresh() if window else None
    for stt in Storage_Types:
        PromedioMT = "PromedioM_"+ stt
        IngressosT = "Ingressos($)_"+ stt
        clientes_tarifas_base= df[(df.Facility==f)&(df["IceStar Contract Type"]==Storage_Types_Dict[stt])].groupby(['Client_Code',"Moneda_Geral"]).apply(lambda x: x["Valor"].mean()).to_dict()
        try:
            if not clientes_tarifas_base['Facility']:
                break
        except:
            pass
        print(stt)

Running storage calculation for: Quilicura
Frozen
Running storage calculation for: Buin
Frozen


In [51]:
PromedioMT = "PromedioM_"+ stt
IngressosT = "Ingressos($)_"+ stt
clientes_tarifas_base= df[(df.Facility==f)&(df["IceStar Contract Type"]==Storage_Types_Dict[stt])].groupby(['Client_Code',"Moneda_Geral"]).apply(lambda x: x["Valor"].mean()).to_dict()
clientes_TO= dftp[(dftp.Facility==f)&(dftp.ContractType =="TakeorPay")].Client_Code.unique()
for c in clientes_TO:
    checkvalue=1
    for cli in clientes_tarifas_base:
        if cli[0]== c:
            clientes_tarifas_base[cli] = dftp[dftp.Client_Code ==c].Rate.mean()
            checkvalue=0
    if checkvalue:
        clientes_tarifas_base[(c,"PESOS")] = dftp[dftp.Client_Code ==c].Rate.mean()

In [52]:
m=1
countm = {}
actualmonth = {cm[0]:0 for cm in clientes_tarifas_base}
while m <=12:
    clientes_tarifas={}
    for cm in clientes_tarifas_base:
        c=cm[0]
        if c!=0:
            if cm[1]=="UF":
                if clientes_tarifas_base[cm] <1:
                    if m >= currentMonth:
                        clientes_tarifas[cm]=clientes_tarifas_base[cm]*ufs[ufs.Mes==(currentMonth-1)].Total.mean()
                    else:
                        clientes_tarifas[cm]=clientes_tarifas_base[cm]*ufs[ufs.Mes==m].Total.mean()
                else:
                    clientes_tarifas[cm]=clientes_tarifas_base[cm]
            else:
                clientes_tarifas[cm]=clientes_tarifas_base[cm]   
            rate= clientes_tarifas[cm]
            revenue= dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Revenue.sum()
            if (cm[1]=="PESOS")|(cm[1]=="UF"):
                initialm=dftp[(dftp.Facility==f)&(dftp.Client_Code== c)]['Rate Adjustment Initial Month'].mean()
                frequency= dftp[(dftp.Facility==f)&(dftp.Client_Code== c)]['Rate Adjustment Frequency'].mean()
                inflation = dproj[(dproj.Facility==f)&(dproj.Client_Code == c)].Inflation.mean()
                if frequency == 12:
                    rate= clientes_tarifas[cm] 
                    revenue= dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Revenue.sum()
                elif m == initialm:
                    rate = (clientes_tarifas[cm]*(1+inflation)**(m))
                    revenue= (dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Revenue.sum()*(1+inflation)**m)
                    actualmonth[c] = initialm + frequency
                    countm[c]= 1
                elif frequency == 1:
                    rate = (clientes_tarifas[cm]*(1+inflation)**(m))
                    revenue= (dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Revenue.sum()*(1+inflation)**(m))
                elif m == actualmonth[c]:
                    countm[c]+=1
                    rate = (clientes_tarifas[cm]*(1+inflation)**(m*countm[c]))
                    revenue= (dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Revenue.sum()*(1+inflation)**(m*countm[c]))
                if c in clientes_TO:
                    dif_positions= (dproj[(dproj.Facility==f)&(dproj.Client_Code == c)&(dproj.Mes == m)][PromedioMT].mean()-dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Positions.sum())
                    if (dftp[(dftp.Client_Code== c)].Type.unique()[0]=="Positions"):
                        if dif_positions >=0:
                            var_rate= dftp[dftp.Client_Code ==c]["Variable Positions Rate"].mean()
                            dproj.loc[(dproj.Facility==f)&(dproj.Client_Code == c)& (dproj.Mes == m), IngressosT] = rate *dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Positions.sum()*monthrange(currentYear,m)[1]+var_rate * dif_positions * monthrange(currentYear,m)[1]
                        else:
                            dproj.loc[(dproj.Facility==f)&(dproj.Client_Code == c)& (dproj.Mes == m), IngressosT] = rate *dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Positions.sum()*monthrange(currentYear,m)[1]
                    elif (dftp[(dftp.Client_Code== c)].Type.unique()[0]=="Revenue"):
                        if dif_positions >=0:
                            dproj.loc[(dproj.Facility==f)&(dproj.Client_Code == c)& (dproj.Mes == m), IngressosT] = revenue + rate * dif_positions * monthrange(currentYear,m)[1]
                        else:
                            dproj.loc[(dproj.Facility==f)&(dproj.Client_Code == c)& (dproj.Mes == m), IngressosT] = revenue
                else:
                    dproj.loc[(dproj.Facility==f)&(dproj.Client_Code == c)& (dproj.Mes == m), IngressosT] = rate * dproj[(dproj.Facility==f)&(dproj.Client_Code == c)&(dproj.Mes == m)][PromedioMT].mean() * monthrange(currentYear,m)[1]
    m+=1

In [53]:
dproj = dproj.drop_duplicates()
clientes_TO= dftp[(dftp.ContractType =="TakeorPay")].Client_Code.unique()
# Ingessos BLAST FREEZING (Blast Freezing Calculation)
clientes_tarifas_base= df[(df.Facility==f)&(df["IceStar Contract Type"]=='Blast Freezing')].groupby(['Client_Code',"Moneda_Geral"]).apply(lambda x: x["Valor"].mean()).to_dict()
m=1
countm = {}
actualmonth = {cm[0]:0 for cm in clientes_tarifas_base}
while m <=12:
    clientes_tarifas={}
    for cm in clientes_tarifas_base:
        c=cm[0]
        if cm[1]=="UF":
            if clientes_tarifas_base[cm] <1:
                if m >= currentMonth:
                    clientes_tarifas[cm]=clientes_tarifas_base[cm]*ufs[ufs.Mes==(currentMonth-1)].Total.mean()
                else:
                    clientes_tarifas[cm]=clientes_tarifas_base[cm]*ufs[ufs.Mes==m].Total.mean()
            else:
                clientes_tarifas[cm]=clientes_tarifas_base[cm]   
        else:
            clientes_tarifas[cm]=clientes_tarifas_base[cm]   
        rate= clientes_tarifas[cm] 
        revenue= dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Revenue.sum()
        if (cm[1]=="PESOS")|(cm[1]=="UF"):
            initialm=dftp[(dftp.Facility==f)&(dftp.Client_Code== c)]['Rate Adjustment Initial Month'].mean()
            frequency= dftp[(dftp.Facility==f)&(dftp.Client_Code== c)]['Rate Adjustment Frequency'].mean()
            inflation = dproj[(dproj.Facility==f)&(dproj.Client_Code == c)].Inflation.mean()
            if frequency == 12:
                rate= clientes_tarifas[cm] 
                revenue= dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Revenue.sum()
            elif m == initialm:
                rate = (clientes_tarifas[cm]*(1+inflation)**(m))
                revenue= (dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Revenue.sum()*(1+inflation)**m)
                actualmonth[c] = initialm + frequency
                countm[c]= 1
            elif m == actualmonth[c]:
                countm[c]+=1
                rate = (clientes_tarifas[cm]*(1+inflation)**(m*countm[c]))
                revenue= (dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Revenue.sum()*(1+inflation)**(m*countm[c]))
            elif frequency == 1:
                rate = (clientes_tarifas[cm]*(1+inflation)**(m))
                revenue= (dftp[(dftp.Facility==f)&(dftp.Client_Code== c)].Revenue.sum()*(1+inflation)**(m))
            dproj.loc[(dproj.Facility==f)&(dproj.Client_Code == c)& (dproj.Mes == m), "BLAST FREEZING"] = rate * dproj[(dproj.Facility==f)&(dproj.Client_Code == c)&(dproj.Mes == m)]['Blast Freezing Volume'].sum() 
    m+=1  

In [54]:
# Por servicio
dproj=dproj.fillna(0)
dfat= dfinal[dfinal.Type== "ACT_" + str(currentYear)]
dfat=dfat.rename(columns={'Month': "Mes"})
dfat=dfat[dfat.Mes!=0]
meses= list(range(1,13))
clientes= dfat.Client_Code.unique()
m = currentMonth - 1
servs= dprojservs.dropna().Specified_Services.unique()
sems_meses= dproj.groupby(['Mes']).apply(lambda x: len(x["Semana"].unique())).to_dict()
dfat=dfat.fillna(0)
count=0

In [56]:
clientes= dfat[dfat.Facility==f].Client_Code.unique()
mesflag={c:0 for c in clientes}
mesvalor={c:0 for c in clientes}
for c in clientes:
    for me in meses:
        Total= dfat[(dfat.Facility == f)&(dfat.Client_Code == c) &(dfat.Mes == me) & (dfat.Service == "STORAGE")].Total.sum()
        if Total!= 0:
            for s in servs:
                strexp= 'Expected_'+ s+' (0 or 1)'
                if dproj[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me)][strexp].max()==1:
                    mesflag[c]=1
                    mesvalor=dproj[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me)&(dproj[strexp]!=0)][s].mean()
                    dproj.loc[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me),s]= mesvalor
                else:
                    if mesflag[c]:
                        dproj.loc[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me),s]= mesvalor
                    else:
                        dproj.loc[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me),s] = dfat[(dfat.Facility == f)&(dfat.Client_Code == c) &(dfat.Mes == me)&(dfat.Service == s)].Total.sum()/Total
            dproj.loc[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me),"STORAGE"] = 1
        else:
            if me>= currentMonth:
                m= currentMonth-1
                Total =dfat[(dfat.Facility == f)&(dfat.Client_Code == c) &(dfat.Mes <=m) & (dfat.Service == "STORAGE")].Total.sum()
                if not Total !=0:
                    Total= dfat[(dfat.Facility == f)&(dfat.Mes <= m)].Total.sum()
            else:
                Total= dfat[(dfat.Facility == f)&(dfat.Mes == me)].Total.sum()
                m=me
            for s in servs:
                strexp= 'Expected_'+s+' (0 or 1)'
                if dproj[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me)][strexp].max()==1:
                    mesflag[c]=1
                    mesvalor=dproj[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me)&(dproj[strexp]!=0)][s].mean()
                else:
                    if mesflag[c]:
                        dproj.loc[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me),s]= mesvalor
                    else:
                        dproj.loc[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me),s] = dfat[(dfat.Facility == f)&(dfat.Client_Code == c)&(dfat.Mes <= m)&(dfat.Service == s)].Total.sum()/Total
                dproj.loc[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me),"STORAGE"] = 1 

clientes= dfat[dfat.Facility==f].Client_Code.unique()
clientes_budg=dproj[(~dproj.Client_Code.isin(clientes)) & (dproj.Facility==f)].Client_Code.unique()
mesflag={c:0 for c in clientes_budg}
mesvalor={c:0 for c in clientes_budg}
for c in clientes_budg:
    for me in meses:
        if me>= currentMonth:
            Total= dfat[(dfat.Facility == f)&(dfat.Mes <= currentMonth-1)].Total.sum()
            m= currentMonth-1
        else:
            Total= dfat[(dfat.Facility == f)&(dfat.Mes == me)].Total.sum()
            m=me
        for s in servs:
            strexp= 'Expected_'+ s+' (0 or 1)'
            if dproj[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me)][strexp].max()==1:
                mesflag[c]=1
                mesvalor=dproj[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me)][s].mean()
            else:
                if mesflag[c]:
                        dproj.loc[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me),s]= mesvalor
                else:
                    dproj.loc[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me),s] = dfat[(dfat.Facility == f)&(dfat.Client_Code == c) &(dfat.Mes <= m)&(dfat.Service == s)].Total.sum()/Total
        dproj.loc[(dproj.Facility == f)&(dproj.Client_Code ==c) & (dproj.Mes ==me),"STORAGE"] = 1 

In [191]:
dproj['STORAGE']=1
dproj['OTHERS'] =dproj.OTHERS*(dproj['Ingressos($)_Frozen'])
dproj['HANDLING'] =dproj.HANDLING*dproj['Ingressos($)_Frozen']
dproj['STORAGE'] =dproj.STORAGE*(dproj['Ingressos($)_Frozen']+dproj['Ingressos($)_Refrigerated']+dproj['Ingressos($)_Dry'])
dprojteste=dproj[dproj.Mes>(currentMonth - 1)].copy()
# Columns Organization
dtiposconts= dftp[['Client_Code','ContractType']].drop_duplicates()
dprojteste= pd.merge(dprojteste,dtiposconts,on ='Client_Code',how ='outer')
dprojteste=dprojteste.drop_duplicates()
dprojteste= dprojteste.groupby(['Mes','Client_Code','ContractType','Ingressos($)_Frozen',"Facility","BLAST FREEZING",'OTHERS','HANDLING']).apply(lambda x: x[['STORAGE']].mean()).reset_index()
dprojteste.loc[(dprojteste.Client_Code.isin(clientes_TO)),'ContractType']="TakeorPay"
dprojteste=dprojteste.drop_duplicates()
dprojteste["Type"]="YTG"

In [192]:
dproj.head()

,Facility,Client_Code,Client_Name,Mes,Semana,check,Inflation,In(Frozen),Out(Frozen),Balance(Frozen),MesSaldo,SemanaSaldo,Saldo.0,In(Refrigerated),Out(Refrigerated),Balance(Refrigerated),MesSaldo2,SemanaSaldo3,Saldo.04,In(Dry),Out(Dry),Balance(Dry),MesSaldo8,SemanaSaldo9,Saldo.010,Blast Freezing Volume,Expected_HANDLING (0 or 1),HANDLING,Expected_PICKING (0 or 1),PICKING,Expected_LABELING (0 or 1),LABELING,Expected_TRANSPORTATION (0 or 1),TRANSPORTATION,Expected_OTHERS (0 or 1),OTHERS,PromedioM_Frozen,PromedioM_Refrigerated,PromedioM_Dry,Ingressos($)_Frozen,Ingressos($)_Refrigerated,Ingressos($)_Dry,BLAST FREEZING,STORAGE
0,Quilicura,1,AGRICOLA ARIZTIA,1,1,False,0.00,31.00,0.00,31,0,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Quilicura,1,AGRICOLA ARIZTIA,1,2,False,0.00,0.00,29.00,2,1,1,0.00,0.00,0.00,0,1,1,0.00,0.00,0.00,0,1,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Quilicura,1,AGRICOLA ARIZTIA,1,3,False,0.00,0.00,0.00,2,1,2,0.00,0.00,0.00,0,1,2,0.00,0.00,0.00,0,1,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Quilicura,1,AGRICOLA ARIZTIA,1,4,False,0.00,40.00,0.00,42,1,3,0.00,0.00,0.00,0,1,3,0.00,0.00,0.00,0,1,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Quilicura,1,AGRICOLA ARIZTIA,2,1,False,0.00,102.00,0.00,144,1,4,0.00,0.00,0.00,0,1,4,0.00,0.00,0.00,0,1,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [193]:
dproj = dprojteste.copy()

In [194]:
dproj.head()

,Mes,Client_Code,ContractType,Ingressos($)_Frozen,Facility,BLAST FREEZING,OTHERS,HANDLING,STORAGE,Type
0,3,1,Pallet per Day,0.00,Quilicura,0.00,0.00,0.00,0.00,YTG
1,3,4,Pallet per Day,0.00,Quilicura,0.00,0.00,0.00,0.00,YTG
2,3,5,Pallet per Day,0.00,Quilicura,0.00,0.00,0.00,0.00,YTG
3,3,6,Pallet per Day,0.00,Quilicura,0.00,0.00,0.00,0.00,YTG
4,3,7,Pallet per Day,0.00,Quilicura,0.00,0.00,0.00,0.00,YTG


In [195]:
dproj[dproj['Client_Code']==44]['STORAGE'].sum()

0.0

In [196]:
# Pipeline Database
print("Reading Pipeline Database")
#window.refresh() if window else None
arq_p = pd.ExcelFile('Databases/Pipeline_Database_CRM.xlsx')
dpip= pd.read_excel(arq_p,"Pipeline Database")
dpip['Operation Beginning Month'] = pd.DatetimeIndex(dpip['Operation Beginning Date']).month
dpip['Operation Beginning Day'] = pd.DatetimeIndex(dpip['Operation Beginning Date']).weekday
dpip['Operation Beginning Year'] = pd.DatetimeIndex(dpip['Operation Beginning Date']).year
dpip.rename(columns={'Account Name (Client)':'Client'},inplace=True)
colunas = dpip.columns
dpip= dpip[(dpip['Sales Pipeline Stages']!=  '6. Closed - Won') & (dpip['Sales Pipeline Stages']!= '6. Closed - Lost') & (dpip['Probability (edit)']>=0.9)]
for c in dpip.Client.unique():
    try:
        mes=dpip[(dpip.Client==c)]['Operation Beginning Month'].unique()[0]
        dia=dpip[(dpip.Client==c)]['Operation Beginning Day'].unique()[0]
        ano=dpip[(dpip.Client==c)]['Operation Beginning Year'].unique()[0]
        if ano == currentYear:
            valor=dpip[(dpip.Client==c)]['Monthly Estimated Revenue (CLP$)'].mean()* dpip[(dpip.Client==c)]['Probability (edit)'].mean()
            dur=dpip[(dpip.Client==c)]['Contract Duration (Months)'].unique()[0]
            if not math.isnan(mes):
                mes=int(mes)
                dia=int(dia)
                i=mes
                count=0
                mrange= monthrange(currentYear,mes)
                if dia>1:
                    valor = valor/mrange[1]*(mrange[1]-dia)
                while i <=12:
                    if count > dur:
                        break
                    dpip.loc[(dpip.Client==c),i]= valor
                    i+=1
                    count+=1
    except:
        print(c)
        #window.refresh() if window else None        
dpip=dpip.melt(id_vars=colunas, var_name="Mes", value_name="Total")
dpip['Total'] = dpip['Total'].astype(float)
dpipservs= pd.read_excel(arq_p,sheet_name="Data Validation")
dpipservs=dpipservs[["Services"]]
dpipservs= dpipservs.dropna()
servispipe= dpipservs.Services.unique()
for s in servispipe:
    sp="%"+s
    dpip[s]= dpip[sp] * dpip["Total"]
dpip.rename(columns={'Client Segment': "Segment"}, inplace=True)
dpip.rename(columns={'Contract Type': "ContractType"}, inplace=True)
dpip=dpip[[ 'Client','Client_Code','GroupCode','Facility',"ContractType",'Segment','Mes','Sales Pipeline Stages','Probability (edit)','Operation Beginning Date','Opportunity Opening Date','STORAGE','HANDLING','BLASTFREEZING','OTHERS']]
dpip["Type"]="Pipeline"
m= currentMonth-1
dpipfinal= dpip[dpip.Mes>m]
dpip=dpipfinal
dpip = dpip.melt(id_vars=['Facility',"Client",'Client_Code','GroupCode', "Mes","Type","ContractType", 'Segment','Sales Pipeline Stages','Probability (edit)','Operation Beginning Date','Opportunity Opening Date'], var_name="Service", value_name="Total")
dpip.rename(columns={'Client': 'Client_Name'}, inplace=True)
dpip.rename(columns={'ContractType': 'Client_Type'}, inplace=True)
#dpip.to_excel("Databases/BIinputs/Pipeline_Database.xlsx",index=False)
dpip = dpip[['Facility',"Client_Name","Client_Code","GroupCode", "Mes","Type","Client_Type", 'Segment','Service','Total']]

Reading Pipeline Database


In [197]:
dpip.sort_values(['Client_Name','Mes']).head(12)

,Facility,Client_Name,Client_Code,GroupCode,Mes,Type,Client_Type,Segment,Service,Total
42,Quilicura,AXIONLOG,NaN,NaN,3,Pipeline,Pallet per day,Food Service,STORAGE,NaN
92,Quilicura,AXIONLOG,NaN,NaN,3,Pipeline,Pallet per day,Food Service,HANDLING,NaN
142,Quilicura,AXIONLOG,NaN,NaN,3,Pipeline,Pallet per day,Food Service,BLASTFREEZING,NaN
192,Quilicura,AXIONLOG,NaN,NaN,3,Pipeline,Pallet per day,Food Service,OTHERS,NaN
47,Quilicura,AXIONLOG,NaN,NaN,4,Pipeline,Pallet per day,Food Service,STORAGE,NaN
97,Quilicura,AXIONLOG,NaN,NaN,4,Pipeline,Pallet per day,Food Service,HANDLING,NaN
147,Quilicura,AXIONLOG,NaN,NaN,4,Pipeline,Pallet per day,Food Service,BLASTFREEZING,NaN
197,Quilicura,AXIONLOG,NaN,NaN,4,Pipeline,Pallet per day,Food Service,OTHERS,NaN
2,Quilicura,AXIONLOG,NaN,NaN,5,Pipeline,Pallet per day,Food Service,STORAGE,"9,668,115.69"
52,Quilicura,AXIONLOG,NaN,NaN,5,Pipeline,Pallet per day,Food Service,HANDLING,"2,578,164.18"


In [198]:
# Final Database
dprojfinal=dproj[['Client_Code','STORAGE','HANDLING','OTHERS','BLAST FREEZING','Mes','Type',"Facility","ContractType"]]
dprojfinal = dprojfinal.fillna(0)
dprojfinal=dprojfinal.drop_duplicates()
dprojfinal = dprojfinal[['Client_Code', 'STORAGE', 'HANDLING', 'OTHERS', 'Mes', 'Type', 'Facility','BLAST FREEZING']]
dprojfinal = dprojfinal.melt(id_vars=["Client_Code", "Mes","Type",'Facility'], var_name="Service", value_name="Total")
dprojfinal.Client_Code= dprojfinal.Client_Code.replace(np.nan, 0)
dprojfinal.Client_Code = dprojfinal.Client_Code.astype(int)
dclientes_tot= pd.read_excel(arq,'Icestar_Clients')[['Client_Code', 'Client_Name']]
dprojfinal= pd.merge(dprojfinal,dclientes_tot,on ='Client_Code',how ='left')

In [199]:
dprojfinal.head()

,Client_Code,Mes,Type,Facility,Service,Total,Client_Name
0,1,3,YTG,Quilicura,STORAGE,0.00,AGRICOLA ARIZTIA
1,4,3,YTG,Quilicura,STORAGE,0.00,AGROSUPER COMERCIAL
2,5,3,YTG,Quilicura,STORAGE,0.00,ELABORADORA DOÑIHUE
3,6,3,YTG,Quilicura,STORAGE,0.00,FAENADORA LO MIRANDA
4,7,3,YTG,Quilicura,STORAGE,0.00,FAENADORA SAN VICENTE


In [200]:
dprojfinal[dprojfinal['Client_Code']==44]['Total'].sum()

0.0

In [201]:
# Organizacao base servicos (Services Database Organization)
dfinal.rename(columns={'Month': "Mes"}, inplace=True)
dfinal.Client_Code.unique()
dfinal.Client_Code= dfinal.Client_Code.replace(np.nan, 0)
dfinal.Client_Code = dfinal.Client_Code.astype(int)
dfinal=pd.concat([dfinal,dprojfinal])
dfinal.drop('ContractType',axis=1,inplace=True)
dclientes= pd.read_excel(arq,'Icestar_Clients')[['Client_Code','Segment','Group','GroupCode','Client_Type','Operation_Type']]
dclientes.Client_Code = dclientes.Client_Code.astype(int)
dfinal.Client_Code.unique()
dfinal.Client_Code= dfinal.Client_Code.replace(np.nan, 0)
dfinal.Client_Code = dfinal.Client_Code.astype(int)
dfinal= pd.merge(dfinal,dclientes,on ='Client_Code',how ='left')
dfinal=pd.concat([dfinal,dpip])
dfinal['Segment'].fillna("OTHER",inplace=True)


In [203]:
dfinal.head()

,Facility,Client_Name,Client_Code,Service,Mes,Total,Type,Segment,Group,GroupCode,Client_Type,Operation_Type
0,Quilicura,AGRICOLA ARIZTIA,1.00,STORAGE,2,"9,688,280.92",Budget,Protein,AGRICOLA ARIZTIA,1.00,Regular,NaN
1,Quilicura,AGRICOLA ARIZTIA,1.00,STORAGE,3,"10,756,702.24",Budget,Protein,AGRICOLA ARIZTIA,1.00,Regular,NaN
2,Quilicura,AGRICOLA ARIZTIA,1.00,STORAGE,4,"10,787,179.56",Budget,Protein,AGRICOLA ARIZTIA,1.00,Regular,NaN
3,Quilicura,AGRICOLA ARIZTIA,1.00,STORAGE,5,"10,817,743.23",Budget,Protein,AGRICOLA ARIZTIA,1.00,Regular,NaN
4,Quilicura,AGRICOLA ARIZTIA,1.00,STORAGE,6,"10,498,445.33",Budget,Protein,AGRICOLA ARIZTIA,1.00,Regular,NaN


dfinal[(dfinal['Client_Code']==44)&(dfinal['Type'].isin(['YTG','ACT_2024']))].groupby('Service')['Total'].sum()

# Base de dados conversao dolar
arq_rates= pd.ExcelFile('Databases/External/Dollar_rates.xlsx')
rates = pd.read_excel(arq_rates)
rates_dict = dict(zip(rates['Facility'],rates['Rate']))
rates_dict[np.nan] = 0
dfinal['Total Local'] = dfinal['Total'].copy()
dfinal['Total'] = dfinal.apply(lambda x: x['Total']*rates_dict[x['Facility']],axis = 1)